In [1]:
import os
import subprocess
from pathlib import Path

Downloading the data

In [2]:
with open('data/experimental/chipexo_conditions/E-MTAB-6673.sdrf.txt','r') as tsv:
    for line in tsv:
        tokens = line.strip().split('\t')
        if "Cbf1" in tokens[0] or "Tye7" in tokens[0]:
            print(tokens[0].replace(" ", "_"), tokens[-3], tokens[-5])
            Path("/oak/stanford/groups/akundaje/amr1/pho4/"+tokens[0].replace(" ", "_")).mkdir(parents=True, exist_ok=True)
            cmd = ['wget', tokens[-3], '-O',
                   "/oak/stanford/groups/akundaje/amr1/pho4/"+tokens[0].replace(" ", "_")+"/"+tokens[-5]]
            subprocess.Popen(cmd).wait()

Cbf1_Ana_1 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/004/ERR2529254/ERR2529254.fastq.gz Cbf1A1_S3_L001_R1_001.fastq.gz
Cbf1_Ana_1 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/005/ERR2529255/ERR2529255.fastq.gz Cbf1A1_S3_L002_R1_001.fastq.gz
Cbf1_Ana_1 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/006/ERR2529256/ERR2529256.fastq.gz Cbf1A1_S3_L003_R1_001.fastq.gz
Cbf1_Ana_1 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/007/ERR2529257/ERR2529257.fastq.gz Cbf1A1_S3_L004_R1_001.fastq.gz
Cbf1_Ana_2 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/008/ERR2529258/ERR2529258.fastq.gz Cbf1A2_S5_L001_R1_001.fastq.gz
Cbf1_Ana_2 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/009/ERR2529259/ERR2529259.fastq.gz Cbf1A2_S5_L002_R1_001.fastq.gz
Cbf1_Ana_2 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/000/ERR2529260/ERR2529260.fastq.gz Cbf1A2_S5_L003_R1_001.fastq.gz
Cbf1_Ana_2 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR252/001/ERR2529261/ERR2529261.fastq.gz Cbf1A2_S5_L004_R1_001.fastq.gz
Tye7_Ana_1 ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ER

In [3]:
!head -n3 ddata/experimental/chipexo_conditions/itions/E-MTAB-6673.sdrf.txt

Source Name	Comment[ENA_SAMPLE]	Comment[BioSD_SAMPLE]	Characteristics[organism]	Characteristics[strain]	Characteristics[growth condition]	Characteristics[genotype]	Material Type	Protocol REF	Protocol REF	Protocol REF	Protocol REF	Protocol REF	Extract Name	Comment[LIBRARY_LAYOUT]	Comment[LIBRARY_SELECTION]	Comment[LIBRARY_SOURCE]	Comment[LIBRARY_STRAND]	Comment[LIBRARY_STRATEGY]	Protocol REF	Performer	Assay Name	Comment[technical replicate group]	Technology Type	Comment[ENA_EXPERIMENT]	Scan Name	Comment[SUBMITTED_FILE_NAME]	Comment[ENA_RUN]	Comment[FASTQ_URI]	Factor Value[environmental stress]	Factor Value[genotype]
Cat8 Ana 1	ERS2411191	SAMEA4590472	Saccharomyces cerevisiae	CEN.PK-113-5D KiURA3	synthetic media in chemostat	CAT8-TAP	whole organism	P-MTAB-73878	P-MTAB-73882	P-MTAB-73883	P-MTAB-73879	P-MTAB-73880	Cat8 Ana 1	SINGLE	ChIP	GENOMIC	not applicable	ChIP-Seq	P-MTAB-73881	DTU	Cat8A1_S5_L001	group 1	sequencing assay	ERX2548083	Cat8A1_S5_L001_R1_001.fastq.gz	Cat8A1_S5_L001_R1_001.fa

In [4]:
!ls /oak/stanford/groups/akundaje/amr1/pho4/

Cbf1_Ana_1		  Cbf1_Glu_2		       Tye7_Ana_1
Cbf1_Ana_2		  Cbf1_Nit_1		       Tye7_Ana_2
Cbf1_ChIP-exo_replicate1  Cbf1_Nit_2		       Tye7_Eth_1
Cbf1_ChIP-exo_replicate2  Cbf1_PB-exo_1uM_replicate1   Tye7_Eth_2
Cbf1_ChIP-exo_replicate3  Cbf1_PB-exo_1uM_replicate2   Tye7_Glu_1
Cbf1_ChIP-exo_replicate4  NoAb_ChIP-exo_replicate1     Tye7_Glu_2
Cbf1_ChIP-exo_replicate5  NoAb_ChIP-exo_replicate2     Tye7_Nit_1
Cbf1_Eth_1		  NoAb_PB-exo_replicate1       Tye7_Nit_2
Cbf1_Eth_2		  Pho4_PB-exo_50nM_replicate1
Cbf1_Glu_1		  Pho4_PB-exo_50nM_replicate2


Combining the data across lanes

In [1]:
%%bash
for filename in /oak/stanford/groups/akundaje/amr1/pho4/*; do
    if [[ $filename != *"replicate"* ]]; then
        zcat $filename/*L001_R1_001.fastq.gz $filename/*L002_R1_001.fastq.gz $filename/*L003_R1_001.fastq.gz $filename/*L004_R1_001.fastq.gz | gzip -c > $filename/sample_R1_001.fastq.gz
        rm $filename/*L00*.fastq.gz
    fi
done

Converting TSV peak files to beds

In [2]:
chrms = ["chrI","chrII","chrIII","chrIV","chrV","chrVI","chrVII","chrVIII",
         "chrIX","chrX","chrXI","chrXII","chrXIII","chrXIV","chrXV","chrXVI","chrM"]

In [3]:
!head data/experimental/limited_conditions/GEMout_Tye7.tsv

Chr	Pos	GeneSys	GeneCom	Nit	Eth	Oxy	Glu	Motif
12	213697	YLR044C	PDC1	5.80933852140078	609.87145557656	4.67816091954023	155.722222222222	--------
7	1026837	YGR254W	ENO1	20.3368700265252	416.356470588235	24.6666666666667	257.121301775148	--------
8	451338	YHR174W	ENO2	4.2962962962963	464.358181818182	14.3670886075949	133.173553719008	--------
11	333871	YKL060C	FBA1	14.2426470588235	179.040780141844	80.9401709401709	289.175324675325	--------
11	333833	YKL060C	FBA1	4.48073394495413	336.733156028369	0.254237288135593	216.062770562771	CATGCTTT
3	142574	YCR011C	ADP1	10.6687631027254	257.796261682243	33.4561403508772	254.482213438735	--------
3	142574	YCR012W	PGK1	10.6687631027254	257.796261682243	33.4561403508772	254.482213438735	--------
10	444994	YJR009C	TDH2	14.1344086021505	382.097690941385	3.20212765957447	140.719230769231	--------
4	542959	YDR050C	TPI1	0	322.627383015598	5.98837209302326	158.890151515152	--------


In [4]:
firstLine = True
tye7_out = open("data/experimental/limited_conditions/unsorted.GEMout_Tye7.bed", "a")
with open("data/experimental/limited_conditions/GEMout_Tye7.tsv") as inp:
    for line in inp:
        if firstLine:
            firstLine = False
            continue
        chrm = chrms[int(line.strip().split('\t')[0])-1]
        pos = int(line.strip().split('\t')[1])
        tye7_out.write(str(chrm)+"\t"+str(pos)+"\t"+str(pos+1)+"\t.\n")
tye7_out.close()

In [5]:
!sort -V -k1,1 -k2,2 data/experimental/limited_conditions/unsorted.GEMout_Tye7.bed | uniq > data/experimental/limited_conditions/GEMout_Tye7.bed

In [6]:
!head data/experimental/limited_conditions/GEMout_Tye7.bed

chrI	34672	34673	.
chrI	57567	57568	.
chrI	60486	60487	.
chrI	60696	60697	.
chrI	173132	173133	.
chrI	173200	173201	.
chrII	63818	63819	.
chrII	136360	136361	.
chrII	165393	165394	.
chrII	192125	192126	.


In [7]:
!head data/experimental/limited_conditions/GEMout_Cbf1.tsv

Chr	Pos	GeneSys	GeneCom	Nit	Eth	Oxy	Glu	Motif
15	511323	YOR098C	NUP1	288.643203883495	181.495108464483	384.64623655914	233.852812271731	--------
15	511323	YOR099W	KTR1	288.643203883495	181.495108464483	384.64623655914	233.852812271731	--------
2	662354	YBR222C	PCS60	279.34479717813	223.395823665893	314.670498084291	247.054315789474	GGTCACGTGTAA
4	1448334	YDR510W	SMT3	189.135478408129	218.788684097728	214.61717791411	234.754292254865	ATGTCACGTTTT
12	115900	YLL010C	PSR1	222.645751633987	145.592556317336	262.753468516542	186.900243309002	TGGTCACGTGGTC
12	115900	YLL009C	COX17	222.645751633987	145.592556317336	262.753468516542	186.900243309002	TGGTCACGTGGTC
11	414238	YKL016C	ATP7	201.133027522936	124.04757185332	259.53865652725	159.473122866894	TAGTCACGTGGTC
11	414238	YKL015W	PUT3	201.133027522936	124.04757185332	259.53865652725	159.473122866894	TAGTCACGTGGTC
14	549171	YNL037C	IDH1	118.470372460497	154.814889918888	242.974486180014	187.724268415742	CTATCACGTGATA


In [8]:
firstLine = True
cbf1_out = open("data/experimental/limited_conditions/unsorted.GEMout_Cbf1.bed", "a")
with open("data/experimental/limited_conditions/GEMout_Cbf1.tsv") as inp:
    for line in inp:
        if firstLine:
            firstLine = False
            continue
        chrm = chrms[int(line.strip().split('\t')[0])-1]
        pos = int(line.strip().split('\t')[1])
        cbf1_out.write(str(chrm)+"\t"+str(pos)+"\t"+str(pos+1)+"\t.\n")
cbf1_out.close()

In [9]:
!sort -V -k1,1 -k2,2 data/experimental/limited_conditions/unsorted.GEMout_Cbf1.bed | uniq > data/experimental/limited_conditions/GEMout_Cbf1.bed

In [10]:
!head data/experimental/limited_conditions/GEMout_Cbf1.bed

chrI	20977	20978	.
chrI	21067	21068	.
chrI	24701	24702	.
chrI	34672	34673	.
chrI	34751	34752	.
chrI	34979	34980	.
chrI	50814	50815	.
chrI	60495	60496	.
chrI	89005	89006	.
chrI	89105	89106	.
